In [1]:
from numpy import array
import tensorflow as tf
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [32]:
def avg_relative_error(y, p):
    error = 0
    er = []
    for i in range(len(y)):
        if y[i] != 0 and p[i] != 0:
            e = (((p[i] - y[i]) / max(y[i], p[i])))
            er.append(e[0])
            error += e[0]
    error = error / len(y)
    return abs(error)

In [33]:
countries = ['US','Australia','Austria', 'Brazil', 'China', 'Czechia', 'Denmark', 'France', 'Germany', 'India', 'Italy', 'Korea_South', 'South_Africa', 'Spain', 'UK']
results = []
for country in countries:
    result = {'country':'', 'conf': 0, 'dead': 0, 'recov': 0}
    result['country'] = country
    file = '../Data_Preprocessing/Clean/' + country + '.csv'
    df = pd.read_csv(file)
    confirmed = df['Confirmed_Cases'].tolist()
    recoveries = df['Recoveries'].tolist()
    deaths = df['Deaths'].tolist()

    n_steps = 4

    X, y = split_sequence(confirmed, n_steps)
    X_r, y_r = split_sequence(recoveries, n_steps)
    X_d, y_d = split_sequence(deaths, n_steps)

    n_features = 1
    n_seq = 2
    n_steps = 2
    X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
    X_r = X_r.reshape((X_r.shape[0], n_seq, 1, n_steps, n_features))
    X_d = X_d.reshape((X_d.shape[0], n_seq, 1, n_steps, n_features))
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features), return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=32, kernel_size=(1,1), activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X, y, epochs=500, verbose=0)
    X_test, y_test = split_sequence(confirmed[25:48], 4)
    X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
    res = model.predict(X_test)
    result['conf'] = avg_relative_error(y_test, res)
    
    model.fit(X_r, y_r, epochs=500, verbose=0)
    
    X_test_r, y_test_r = split_sequence(recoveries[25:48], 4)
    X_test_r = X_test_r.reshape((X_test_r.shape[0], n_seq, 1, n_steps, n_features))
    res_r = model.predict(X_test_r)
    result['recov'] = avg_relative_error(y_test_r, res_r)
    
    model.fit(X_d, y_d, epochs=500, verbose=0)
    
    X_test_d, y_test_d = split_sequence(deaths[25:48], 4)
    X_test_d = X_test_d.reshape((X_test_d.shape[0], n_seq, 1, n_steps, n_features))
    res_d = model.predict(X_test_d)
    result['dead'] = avg_relative_error(y_test_d, res_d)
    print(result)
    results.append(result)

results

{'country': 'US', 'conf': 0.3314393678631045, 'dead': 0.27102164532008927, 'recov': 0.14752372315055445}
{'country': 'Australia', 'conf': 0.1890699212488375, 'dead': 0.12253479581130178, 'recov': 0.07400957847896375}
{'country': 'Austria', 'conf': 0.5366294533994637, 'dead': 0.0, 'recov': 0.017156737415414108}
{'country': 'Brazil', 'conf': 0.0009298089303468403, 'dead': 0.0, 'recov': 0.0}
{'country': 'China', 'conf': 0.3184692742008912, 'dead': 0.03898696885689309, 'recov': 0.026872522283443494}
{'country': 'Czechia', 'conf': 0.20927690753811284, 'dead': 0.0, 'recov': 0.0}
{'country': 'Denmark', 'conf': 0.24846672031440234, 'dead': 0.0, 'recov': 0.039904930089649404}
{'country': 'France', 'conf': 0.0710045852159199, 'dead': 0.22857704758644104, 'recov': 0.003716886043548584}
{'country': 'Germany', 'conf': 0.4574022889137268, 'dead': 0.02578266984538028, 'recov': 0.1847235121225056}
{'country': 'India', 'conf': 0.16975452868561997, 'dead': 0.0, 'recov': 0.0}
{'country': 'Italy', 'conf':

[{'country': 'US',
  'conf': 0.3314393678631045,
  'dead': 0.27102164532008927,
  'recov': 0.14752372315055445},
 {'country': 'Australia',
  'conf': 0.1890699212488375,
  'dead': 0.12253479581130178,
  'recov': 0.07400957847896375},
 {'country': 'Austria',
  'conf': 0.5366294533994637,
  'dead': 0.0,
  'recov': 0.017156737415414108},
 {'country': 'Brazil',
  'conf': 0.0009298089303468403,
  'dead': 0.0,
  'recov': 0.0},
 {'country': 'China',
  'conf': 0.3184692742008912,
  'dead': 0.03898696885689309,
  'recov': 0.026872522283443494},
 {'country': 'Czechia',
  'conf': 0.20927690753811284,
  'dead': 0.0,
  'recov': 0.0},
 {'country': 'Denmark',
  'conf': 0.24846672031440234,
  'dead': 0.0,
  'recov': 0.039904930089649404},
 {'country': 'France',
  'conf': 0.0710045852159199,
  'dead': 0.22857704758644104,
  'recov': 0.003716886043548584},
 {'country': 'Germany',
  'conf': 0.4574022889137268,
  'dead': 0.02578266984538028,
  'recov': 0.1847235121225056},
 {'country': 'India', 'conf': 0.1